In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
print(tf.train.Saver.__doc__)

Saves and restores variables.

  See @{$variables$Variables}
  for an overview of variables, saving and restoring.

  The `Saver` class adds ops to save and restore variables to and from
  *checkpoints*.  It also provides convenience methods to run these ops.

  Checkpoints are binary files in a proprietary format which map variable names
  to tensor values.  The best way to examine the contents of a checkpoint is to
  load it using a `Saver`.

  Savers can automatically number checkpoint filenames with a provided counter.
  This lets you keep multiple checkpoints at different steps while training a
  model.  For example you can number the checkpoint filenames with the training
  step number.  To avoid filling up disks, savers manage checkpoint files
  automatically. For example, they can keep only the N most recent files, or
  one checkpoint for every N hours of training.

  You number checkpoint filenames by passing a value to the optional
  `global_step` argument to `save()`:

  ```

In [3]:
mnist = input_data.read_data_sets("data", one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [4]:
# Simple NN
def build_graph(graph):
    with graph.as_default():
        x_ = tf.placeholder(tf.float32, shape=[None, 784], name="x_")
        y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y_")

        W_h = tf.Variable(np.random.randn(784, 100), 
                          dtype=tf.float32, 
                          name="W_h")
        b_h = tf.Variable(np.zeros(100), 
                          dtype=tf.float32,
                          name="b_h")
        z_h = tf.matmul(x_, W_h) + b_h
        layer_h = tf.nn.sigmoid(z_h)

        W_o = tf.Variable(np.random.randn(100, 10),
                          dtype=tf.float32,
                          name="W_o")
        b_o = tf.Variable(np.random.randn(10),
                          dtype=tf.float32,
                          name="b_o")
        z_o = tf.matmul(layer_h, W_o) + b_o
        y = tf.nn.softmax(z_o)
        predict = tf.arg_max(y, 1)

        loss = - tf.reduce_mean(y_*tf.log(y))
        train_op = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

        # saver to save session
        saver = tf.train.Saver() 
    return x_, y_, predict, loss, train_op, saver

In [5]:
def comput_accuracy(y1, y2):
    return 100*(y1 == y2).mean()

In [17]:
batch_size = 200
n_iteration = 20000

graph = tf.Graph()
x_, y_, predict, loss, train_op, saver = build_graph(graph)
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    for step in range(n_iteration):
        train_images, train_labels = mnist.train.next_batch(batch_size)
        feed_dict = {x_: train_images, 
                     y_:train_labels}
        _, l = sess.run([train_op, loss], feed_dict=feed_dict)
    
        if (step+1) % 1000 == 0:
            pred = sess.run(predict, feed_dict={x_:mnist.test.images,
                                                y_:mnist.test.labels})
            acc = comput_accuracy(pred, np.argmax(mnist.test.labels, 1))
            print("Iteration {}: loss {}, acc {}".format(step+1, l, acc))
    saver.save(sess, "models/example_save_session.ckpt")

Iteration 1000: loss 0.24087940156459808, acc 46.01
Iteration 2000: loss 0.15155558288097382, acc 58.220000000000006
Iteration 3000: loss 0.1389244645833969, acc 65.16999999999999
Iteration 4000: loss 0.0990942195057869, acc 69.37
Iteration 5000: loss 0.09231088310480118, acc 72.64
Iteration 6000: loss 0.07886165380477905, acc 74.89
Iteration 7000: loss 0.08633018285036087, acc 76.33
Iteration 8000: loss 0.06446634232997894, acc 77.69
Iteration 9000: loss 0.05860651284456253, acc 78.81
Iteration 10000: loss 0.08533971756696701, acc 79.75999999999999
Iteration 11000: loss 0.055535003542900085, acc 80.61
Iteration 12000: loss 0.07122772186994553, acc 81.19
Iteration 13000: loss 0.056912731379270554, acc 81.96
Iteration 14000: loss 0.06299170851707458, acc 82.52000000000001
Iteration 15000: loss 0.05374893546104431, acc 83.12
Iteration 16000: loss 0.05282098427414894, acc 83.57
Iteration 17000: loss 0.05982606112957001, acc 83.87
Iteration 18000: loss 0.051524125039577484, acc 84.17999999

In [18]:
restore_graph = tf.Graph()
x_, y_, predict, _, _, saver = build_graph(restore_graph)

with tf.Session(graph=restore_graph) as sess:
    tf.global_variables_initializer().run()
    saver.restore(sess, "models/example_save_session.ckpt")
    
    feed_dict = {x_:mnist.test.images,
                 y_:mnist.test.labels}
    pred = sess.run(predict, feed_dict=feed_dict)
    acc = comput_accuracy(pred, np.argmax(mnist.test.labels, 1))
    print("restored accuracy: {}".format(acc))

INFO:tensorflow:Restoring parameters from models/example_save_session.ckpt
restored accuracy: 84.75
